In [ ]:
import os
import openai
import csv
openai.api_key = "" #Enter your API key here
openai.Engine.list()

In [ ]:
model_name = "curie"
openai.Engine.retrieve(model_name)

In [ ]:
#---------- This function gets the dataset and returns a prompt

def data_to_prompt(task_description, train_file_address, test_question):
    prompt = ""
    prompt += task_description + '\n\n'
    
    with open(train_file_address) as train_file:
        train_data = csv.reader(train_file, delimiter=',')
    
        for row in train_data:
            sample_question = row[1]
            sample_answer = row[3]
        
            prompt += "Question:" + sample_question + '\n'
            prompt += "Answer:" + sample_answer + '\n\n'
        
        prompt += "Question:" + test_question + '\n'
        prompt += "Answer:"
    
    return prompt

In [ ]:
train_address = 'Data\\CSV\\PubMed-QA-train-50.csv'
test_address = 'Data\\CSV\\PubMed-QA-test-100.csv'

max_token = 1

test_scores = []
predictions = []

task_description = "It is a question answering task. The answer to every question is yes, maybe, or no."

total_samples = 0
correct_predictions = 0

with open(test_address) as test_file:
    test_data = csv.reader(test_file, delimiter=',')
    
    for row in test_data:
        sample_question = row[1]
        sample_answer = row[3]
        
        prompt = data_to_prompt(task_description, train_address, sample_question)
        response = openai.Completion.create(engine=model_name, prompt=prompt, temperature=0, max_tokens=max_token, top_p=1, frequency_penalty=0, presence_penalty=0, stop=["\n"])
        print(prompt)
        print("*************")
        print(response)
        choices = response["choices"]
        predicted_label = choices[0]
        predicted_label = predicted_label["text"]
        if(len(predicted_label) > 0):
            if(predicted_label[0] == ' '):
                predicted_label = predicted_label[1:]
        print('Test question:', sample_question, ' || Test answer:', sample_answer, ' || Predicted label:', predicted_label)
        print("-----------------------------------------------------------------------------------")
        
        
        total_samples += 1
        if(sample_answer.lower() == predicted_label.lower()):
            correct_predictions += 1

accuracy = correct_predictions / total_samples        
print('Total samples:', total_samples)
print('Correct predictions:', correct_predictions)
print('Accuracy:', accuracy)